# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [29]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [30]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


There might be a problem with your API key? Please visit the troubleshooting notebook!


In [31]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [32]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [33]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [34]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [35]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [36]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [37]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [38]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/magenta-realtime',
 '/nanonets/Nanonets-OCR-s',
 '/google/gemma-3n-E4B-it',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/facebook/seamless-interaction',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammar

In [39]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://huggingface.co/'},
  {'type': 'Company page', 'url': 'https://huggingface.co/allenai'},
  {'type': 'Facebook page', 'url': 'https://www.facebook.com/huggingface/'},
  {'type': 'Intel webpage', 'url': 'https://www.intel.com/'},
  {'type': 'Google Colab page',
   'url': 'https://colab.research.google.com/drive/index.html?usp=sharing'},
  {'type': 'Amazon webpage', 'url': 'https://aws.amazon.com/'},
  {'type': 'Microsoft webpage', 'url': 'https://www.microsoft.com/'},
  {'type': 'Grammarly webpage', 'url': 'https://www.grammarly.com/'},
  {'url': 'https://github.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [40]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        try:
            result += Website(link["url"]).get_contents()
        except Exception as e:
            result += f"Impossibile recuperare la pagina {link['url']}: {e}\n"
    return result

In [41]:
print(get_all_details("https://www.huggingface.co"))

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company/Culture page', 'url': 'https://brand.huggingface.co'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Pricing page', 'url': 'https://endpoints.huggingface.co/pricing#spaces'}, {'type': 'Enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co'}, {'type': 'Research and Development changelog page', 'url': 'https://changelog.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
black-forest-labs/FLUX.1-Kontext-dev
Updated
about 15 hour

In [42]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [43]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [44]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': ' careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Brand page', 'url': 'https://brand.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'Status/Outage page', 'url': 'https://status.huggingface.co/'}, {'type': 'Changelog page', 'url': 'https://changelog.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\nabout 15 hours ago\n•\n12.9k\n•\n817\ntencent/Hunyuan-A13B-Instruct\nUpdated\nabout 10 hours ago\n•\n406\ngoogle/magenta-realtime\nUpdated\n5 days ago\n•\n389\nnanonets/Nanonets-OCR-s\nUpdated\n8 days ago\n•\n202k\n•\n1.22k\ngoogle/gemma-3n-E4B-it\nUpdated\n1 day ago\n•\n5.55k\n•\n218\nBrowse 1M+ models\nSpaces\nRunning\n8.74k\n8.74k\nDeepSite

In [45]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [46]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company/Brand page', 'url': 'https://brand.huggingface.co'}, {'type': 'Research and Collaboration spaces (e.g. GitHub, Discord)', 'url': 'https://discord.cdn.hchq.ae/invite/XM9wRqX'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co'}, {'type': 'News and status updates (Twitter)', 'url': 'https://status.huggingface.co/'}, {'type': 'LinkedIn company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Careers and Join page', 'url': 'https://apply.workable.com/huggingface'}]}


# Welcome to Hugging Face: Where AI Meets Love

Is your data too lonely? Do you need help finding the perfect model? Look no further! At Hugging Face, we're on a mission to bring AI and humans closer together. We believe that with great power comes great responsibility, especially when it comes to machine learning.

## Our Story
We started as a community-driven platform where developers could share and collaborate on models, datasets, and applications. Today, we're proud to be the Home of Machine Learning, with millions of models and datasets at your fingertips.

## Meet Our Community

* Meta: Because who needs to keep track of their own brain?
* Amazon: The AI giant is taking over!
* Google: They know a thing or two about love (and also AI)
* Intel: Who knew CPUs could be so charming? 
* Microsoft: The AI giant's love child... just kidding, it's just the Transformer library
* Grammarly: Because writers need to love themselves too

## What We Do

* **Transformers**: Our state-of-the-art ML library for PyTorch, TensorFlow, and JAX. It's like having a superpower in your browser.
* **Diffusers**: The ultimate guide to diffusion models. Your future self will thank you.
* **Hub**: Our Python client makes it easy to interact with the Hugging Face hub. You're welcome!
* **Datasets**: Over 250k datasets at your disposal, because variety is the spice of life.

## Spaces

Our collaborative platform for model hosting, inference endpoints, and more!

### Running
DeepSite v2: Generate any application with DeepSeek! 🐳
Sparc3D: Next-Gen High-Resolution 3D Model Generation (because who doesn't love a good 3D?)
OmniGen2: Unified Image Understanding and Generation (your future self will thank you)

## Compute, Enterprise & Pricing

We've got paid Compute and Enterprise solutions to help you accelerate your ML journey. Start at $0.60/hour for GPU and go up from there.

### Accelerate
8,875 models to train on your PC! Because why not?

## Changelog & Docs

Want to learn more about our libraries, platforms, or services? Our documentation has got you covered!

[ Learn More ]
[ GitHub ](https://github.com/huggingface)

Stay connected with us:

[ Twitter](https://twitter.com/HuggingFace)
[ Discord](https://discord.gg/huggingface)
[ LinkedIn](https://www.linkedin.com/company/hugging-face/)
 

At Hugging Face, we believe that AI should be loveable too. Join our community and let's build a brighter future together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [47]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [48]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'Company page', 'url': 'https://huggingface.co'}, {'type': 'About page', 'url': 'https://huggingface.co/brand'}, {'type': 'Model webpage', 'url': 'https://huggingface.co/models'}]}


**The Hugging Face Brochure**
==========================

[Cover Image: A cute illustration of a robot giving a hug]

Welcome to Hugging Face, where we're building the future of Artificial Intelligence with love and care.

**Our Mission**
---------------

We're a community-driven platform that brings together passionate individuals from around the world to collaborate on AI models, datasets, and applications. Our mission is to accelerate the development of machine learning, making it more accessible and fun for everyone!

**Who We Are**
--------------

At Hugging Face, we're not just building software – we're building a community of innovators, researchers, and makers. From entrepreneurs to students, our platform welcomes anyone passionate about AI and want to learn or contribute.

**Our Tools**
------------

*   **Models**: Explore over 1 million pre-trained models across various modalities (text, image, video, audio, and 3D).
*   **Datasets**: Browse 250,000+ datasets for any ML task and share your own creations with the world.
*   **Spaces**: Collaborate on unlimited public projects, applications, or even host your own!

**Our Culture**
-------------

Hugging Face is all about warmth, inclusivity, and learning. Our community-driven approach fosters collaboration, support, and growth among members. We're committed to providing a welcoming environment where everyone can thrive.

**Why Join Us?**
----------------

*   **Accelerate Your Career**: Develop your skills in AI, machine learning, and data science with our resources.
*   **Make A Difference**: Contribute to open-source projects, create value for others, and impact the world's future.
*   **Get Involved**: Attend webinars, participate in discussions, and engage with fellow members.

**Join The Hugging Face Family**
--------------------------------

Ready to embark on this exciting journey? Sign up now and become part of our vibrant community!

[Call-to-Action Button: Join Now]

Let's build the future together!

In [49]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co'}, {'type': 'Company/ Brand page', 'url': 'https://huggingface.co_brand'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Models page', 'url': 'https://huggingface.co/models'}]}


#Welcome to Hugging Face: Embracing AI Together

[Cover Image: A group of friends embracing, representing collaboration and community]

At Hugging Face, we believe that machine learning should be a collaborative effort. Our platform is built on the principles of openness, transparency, and community-driven innovation.

**Our Mission**

We're on a mission to build the future of AI by empowering researchers, developers, and innovators from all over the world. Our goal is to provide the tools, resources, and community support necessary for anyone to explore, experiment, and apply AI in their work or projects.

**What We Do**

We offer a comprehensive platform that includes:

- **Models**: Explore over 1 million machine learning models developed by our community.
- **Datasets**: Access and share datasets for any ML tasks with our extensive collection of open-source data.
- **Spaces**: Host, collaborate on, and discover unlimited public models, datasets, and applications in a collaborative environment.

**Join Our Community**

At Hugging Face, we're not just building technology; we're building a community. We welcome:

* Researchers, developers, students, and professionals interested in machine learning.
* Companies interested in exploring AI for their businesses.
* Open-source enthusiasts who contribute to our platform and drive innovation.

### Features

Our features include:

* **Compute**: Optimize your models' deployment with our paid Compute services starting at $0.60/hour for GPU. 
* **Enterprise**: Unlock the full potential of AI with our Enterprise solutions providing advanced security, access controls, and dedicated support.
* **Open Source**: Explore our open-source projects which are the foundation of ML tooling in Python libraries including Transformers, Diffusers, etc.

### Impact

Our impact is real:

- Over 50,000 organizations worldwide trust us for their AI needs
- Our models are being applied to solve critical problems across industries from healthcare AI to social networks

Join us at Hugging Face and experience the thrill of collaborative innovation. Let's build a brighter future together!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>